In [1]:
import torchvision
import torch
from torchvision import transforms
from tqdm.auto import tqdm
import random
import numpy as np
def manual_seed(seed):
    np.random.seed(seed) #1
    random.seed(seed) #2
    torch.manual_seed(seed) #3
    torch.cuda.manual_seed(seed) #4.1
    torch.cuda.manual_seed_all(seed) #4.2
    torch.backends.cudnn.benchmark = False #5 
    torch.backends.cudnn.deterministic = True #6

manual_seed(42)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
pre_vgg16 = torchvision.models.vgg16(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
print(pre_vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

weight에 zero padding 넣기   
weight parameter 변경하기

In [5]:
with torch.no_grad():
    pre_weight = pre_vgg16.features[0].weight.data
    print(pre_weight.shape, type(pre_weight))
    zero_tensor = torch.zeros((64,1,3,3))
    new_weight = torch.concat([pre_weight, zero_tensor], dim=1)
    print(new_weight.shape)
    new_parameter = torch.nn.Parameter(new_weight)
    print(new_parameter.shape)
    new_layer = torch.nn.Conv2d(4,64,3,1,1)
    new_layer.weight = new_parameter
    pre_vgg16.features[0] = new_layer
    print(pre_vgg16.features[0].weight.data.shape)
    print(pre_vgg16)

torch.Size([64, 3, 3, 3]) <class 'torch.Tensor'>
torch.Size([64, 4, 3, 3])
torch.Size([64, 4, 3, 3])
torch.Size([64, 4, 3, 3])
VGG(
  (features): Sequential(
    (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3),

In [6]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomCrop(224,padding=16),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    # transforms.Normalize(mean=(0.5071, 0.4867, 0.4408), std= (0.2675, 0.2565, 0.2761)),
])

test_transform = transforms.Compose([
     transforms.Resize((224,224)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=(0.5071, 0.4867, 0.4408), std= (0.2675, 0.2565, 0.2761)),
])

# train_data = torchvision.datasets.CIFAR100(root="./dataset", train=True, transform=transform)
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
#                                           shuffle=True,pin_memory=True,num_workers=4)
# val_data = torchvision.datasets.CIFAR100(root="./dataset", train=False, transform=test_transform)
# val_loader = torch.utils.data.DataLoader(val_data, batch_size=32,
#                                           shuffle=False,pin_memory=True,num_workers=4)
val_data = torchvision.datasets.ImageNet(root="./dataset/ImageNet", split='val', transform=test_transform)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32,
                                          shuffle=False,pin_memory=True,num_workers=4)


In [7]:
next_data = next(iter(val_loader))[0]
print(next_data.shape)
imgs = torch.cat([next_data, torch.zeros((next_data.size(0),1,224,224))], dim=1)
print(imgs.shape)

torch.Size([32, 3, 224, 224])
torch.Size([32, 4, 224, 224])


In [8]:
pre_vgg16.eval()
pre_vgg16.cuda()
def validate(model, test_loader):
    model.eval()
    val_acc = 0
    with torch.no_grad():
        for data in tqdm(test_loader,leave=True):
            imgs = torch.cat([data[0], torch.zeros((data[0].size(0),1,224,224))], 1)
            imgs, target = imgs.to(device), data[1].to(device)
            output = model(imgs)

            _, preds = torch.max(output.data, 1)
            val_acc += (preds==target).sum().item()
    
    val_acc = 100. * val_acc/len(test_loader.dataset)

    return val_acc

start_acc = validate(pre_vgg16, val_loader)
print(f"START  ACC {start_acc:.2f}")

100%|██████████| 1563/1563 [01:27<00:00, 17.95it/s]

START  ACC 21.81


In [1]:
import torch
from models.vgg import float_vgg16
input_tensor = torch.zeros((64,224,224,4)).cuda()
layer = float_vgg16()
layer.cuda()
with torch.no_grad():
    y = layer(input_tensor)
    print(y.shape)
    # y = y+bias
    # print(y.shape)

torch.Size([64, 100])


In [2]:
import torch
from models.vgg import int_vgg16
input_tensor = torch.zeros((32,224,224,4),dtype=torch.int8).cuda()
layer = int_vgg16()
layer.cuda()
with torch.no_grad():
    y = layer(input_tensor)
    print(y.shape)
    # y = y+bias
    # print(y.shape)

torch.Size([32, 100])
